In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!

import numpy as np
import matplotlib.pyplot as plt

In [5]:
# import os

# # Define the target directory
# target_directory = r"C:\Users\pablosal\Desktop\gbbai-azure-ai-search-indexing"  # change your directory here

# # Check if the directory exists
# if os.path.exists(target_directory):
#     # Change the current working directory
#     os.chdir(target_directory)
#     print(f"Directory changed to {os.getcwd()}")
# else:
#     print(f"Directory {target_directory} does not exist.")

Directory changed to C:\Users\pablosal\Desktop\gbbai-azure-ai-search-indexing


In [9]:
spark

In [ ]:
#%pip install -r builtin/requirements.txt

In [2]:
from dotenv import load_dotenv
import os

# specify the path to your .env file
dotenv_path = 'C:\\Users\\pablosal\\Desktop\\gbbai-azure-ai-fabric-aoai\\.env'

# load the variables
load_dotenv(dotenv_path)

True

In [3]:
from synapse.ml.core.platform import find_secret

service_url = os.getenv('AZURE_AOAI_API_ENDPOINT')
deployment_name = os.getenv('AZURE_AOAI_EMBEDDING_DEPLOYMENT_ID')
key = os.getenv('AZURE_AOAI_API_KEY')

In [4]:
import pyspark
print(pyspark.__version__)

3.4.1


In [5]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from synapse.ml.services import OpenAIEmbedding
from pyspark.sql import SparkSession

In [17]:
import pyspark.sql.functions as F

df = (
    spark.read.options(inferSchema="True", delimiter=",", header=True)
    .csv("wasbs://publicwasb@mmlspark.blob.core.windows.net/fine_food_reviews_1k.csv")
    .repartition(5)
)

df = df.withColumn(
    "combined",
    F.format_string("Title: %s; Content: %s", F.trim(df.Summary), F.trim(df.Text)),
)

display(df.head(5))
df.write.mode("overwrite").format("parquet").save("Files/ " + "fine_food_reviews_1k.parquet")

[Row(id=529, Time=1351209600, ProductId='B0007A0AQW', UserId='A1HHNY1GVD3UIS', Score=5, Summary='my dogs love the peanut butter!', Text="First off, read the ingredients, no crazy words I can't pronounce, which means it's all natural! I got the peanut butter treats for my two children: husky/shepherd and a lab/shepherd mix, and they can't get enough of them. Plus, on Amazon, this is such a great value for 16oz. Beats petsmart for sure! I'll be looking for more Zuke's products now that I've discovered them. Thanks Zuke's!", combined="Title: my dogs love the peanut butter!; Content: First off, read the ingredients, no crazy words I can't pronounce, which means it's all natural! I got the peanut butter treats for my two children: husky/shepherd and a lab/shepherd mix, and they can't get enough of them. Plus, on Amazon, this is such a great value for 16oz. Beats petsmart for sure! I'll be looking for more Zuke's products now that I've discovered them. Thanks Zuke's!"),
 Row(id=132, Time=135

In [6]:
df = spark.read.parquet("Files/ fine_food_reviews_1k.parquet") 
from pyspark.sql import DataFrame

if isinstance(df, DataFrame):
    print("df is a Spark DataFrame")
else:
    print("df is not a Spark DataFrame")

df is a Spark DataFrame


In [ ]:
# importing libraries
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql import Row

import pandas as pd

import openai
import os

import requests
import json

import aiohttp
import asyncio
import nest_asyncio

import ssl

from concurrent.futures import ThreadPoolExecutor, as_completed
from threading import Lock

In [ ]:
# Apply the necessary patch for asyncio to utilise exisiting event loop, if running in Jupyter
nest_asyncio.apply()

# SSL context for self-signed certificate
## Please note that this is an optional parameter if you're not using a self-hosted endpoint
ssl_context = ssl.create_default_context(cafile=certificate_path)

# Function to make asynchronous call to Azure OpenAI API
async def async_query_gpt4(session, api_key, prompt, ssl_context=ssl_context):
    url = api_url  # Adjust as needed
    headers = {
        "Content-Type": "application/json",
        "api-key": f"{api_key}"
    }  
    data = {
        "model": "gpt-4",  # Adjust the model as needed
        "messages": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    }

    async with session.post(url, json=data, headers=headers, ssl=ssl_context) as response:
        return await response.json()

# Function to perform asynchronous tasks with Semaphore for rate limiting
async def async_task(api_key, prompts, ssl_context=ssl_context, max_concurrent_requests=MAX_THREADS):
    async with aiohttp.ClientSession() as session:
        semaphore = asyncio.Semaphore(max_concurrent_requests)
        tasks = []
        responses = []

        async def fetch_and_append(prompt):
            async with semaphore:
                response = await async_query_gpt4(session, api_key, prompt, ssl_context)
                responses.append(response)

        for prompt in prompts:
            task = asyncio.create_task(fetch_and_append(prompt))
            tasks.append(task)
            # You can add logic here to adjust batch sizes adaptively

        await asyncio.gather(*tasks)
        return responses
        
def process_prompts(DF, prompt_col, response_col='response', max_concurrent_requests=MAX_THREADS):

    # create a prompt index and extract prompts into a list
    DF = DF.withColumn("prompt_idx", F.monotonically_increasing_id())

    df = DF[['prompt_idx', prompt_col]].toPandas()
    prompts_idx = df['prompt_idx'].to_list()
    prompts = df[prompt_col].to_list()
    
    # Run async tasks and get results
    loop = asyncio.get_event_loop()
    prompt_responses = loop.run_until_complete(async_task(api_key, prompts, ssl_context, max_concurrent_requests))

    # extracting content from the API responses
    responses = [response['choices'][0]['message']['content'] for response in prompt_responses]

    # converting results into a 
    results = list(zip(prompts_idx, responses))
    results_DF = spark.createDataFrame(results, ["prompt_idx", response_col])

    # Join back with original DataFrame using the index
    DF = DF.join(results_DF, on=['prompt_idx']).drop('prompt_idx')

    return DF
    
# read the dataframe with prompts
responseDF = spark.sql(sql_statement)

# Using pandas converted dataframe to make asynchronous calls
responseDF = process_prompts(DF=responseDF, prompt_col='prompt', response_col="response")

In [7]:
from synapse.ml.services.openai import OpenAIEmbedding

embedding = (
    OpenAIEmbedding()
    .setSubscriptionKey(key)
    .setDeploymentName(deployment_name)
    .setCustomServiceName(service_url)
    .setTextCol("combined")
    .setErrorCol("error")
    .setOutputCol("embeddings")
)

completed_df = embedding.transform(df).cache()
display(completed_df)

TypeError: 'JavaPackage' object is not callable